In [ ]:
import pandas as pd
import ixmp as ix
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In [ ]:
mp = ix.Platform(dbtype='HSQLDB')

In [ ]:
base = message_ix.Scenario(mp, model='Westeros Electrified', 
                               scen='baseline')

In [ ]:
model = 'Westeros Electrified'
scen = base.clone(model, 'renewables','testing new formulation', keep_sol=False)
scen.check_out()

In [ ]:
# re-define useful elements
country = 'Westeros'
model_horizon = [700, 710, 720]
act_years = [690, 700, 710, 720, 700, 710, 720, 710, 720, 720]
vintage_years = [690, 690, 690, 690, 700, 700, 700, 710, 710, 720]

### add sets

In [ ]:
scen.init_set('map_rating',['node','technology','commodity','level','rating','year'])
scen.init_set('map_ren_grade',['node','commodity','grade','year'])

In [ ]:
scen.init_set('year_vtg')
scen.add_set('year_vtg',vintage_years)

In [ ]:
scen.add_set('grade',['g1','g2'])
scen.add_set('rating',['r0','r0_4','r0_8','r_full'])
scen.add_set('level','renewable')
scen.add_set('level_renewable','renewable')
scen.add_set('commodity','wind')

In [ ]:
map_ren_com = pd.DataFrame({
        'node': country,
        'technology' : 'wind_ppl',
        'commodity': 'electricity', # commodity electricity?
        'year': model_horizon,
    })
map_rating_coal = pd.DataFrame({
        'node': country,
        'technology' : 'coal_ppl',
        'commodity': 'electricity',
        'level' : 'final',         # level final?
        'rating' : 'r_full',
        'year': model_horizon,
    })

base_map_rating_wind = pd.DataFrame({
        'node': country,
        'technology' : 'wind_ppl',
        'commodity': 'electricity',
        'level' : 'final',         # level final?
        'year': model_horizon,
    })
base_map_ren_grade = pd.DataFrame({
        'node': country,
        'commodity': 'electricity',
        'year': model_horizon,
    })

In [ ]:
# scen.add_set('map_ren_com',map_ren_com)
scen.add_set('map_rating',map_rating_coal)

map_rating_wind1 = make_df(base_map_rating_wind, rating = 'r0')
scen.add_set('map_rating',map_rating_wind1)
map_rating_wind2 = make_df(base_map_rating_wind, rating = 'r0_4')
scen.add_set('map_rating',map_rating_wind2)
map_rating_wind3 = make_df(base_map_rating_wind, rating = 'r0_8')
scen.add_set('map_rating',map_rating_wind3)

map_ren_grade1 = make_df(base_map_ren_grade, grade = 'g1')
scen.add_set('map_ren_grade',map_ren_grade1)
map_ren_grade2 = make_df(base_map_ren_grade, grade = 'g2')
scen.add_set('map_ren_grade',map_ren_grade2)
# map_ren_grade = map_ren_grade1.append(map_ren_grade2) # it screws thins up with indexes

### add parameters

In [ ]:
# define input for renewable tech with level renewable
b = scen.par('output')
b = b[b['technology'] == 'wind_ppl']
b['commodity'] = 'wind'
b['level'] = 'renewable'
b.columns = scen.par('input').columns
b

In [ ]:
scen.add_par('input',b)

In [ ]:
# renewable_potential(node,commodity,grade,level_renewable,year)
base_renewable_potential = pd.DataFrame({
        'node': country,
        'commodity': 'wind', # or wind?
        'level' : 'renewable',       
        'year': model_horizon,
    })
renewable_potential1 = make_df(base_renewable_potential, grade = 'g1', 
                              value = 2*10**10, unit = 'MW')

scen.add_par('renewable_potential',renewable_potential1)

renewable_potential2 = make_df(base_renewable_potential, grade = 'g2', 
                              value = 2500*10**12, unit = 'MW')
scen.add_par('renewable_potential',renewable_potential2)

In [ ]:
# renewable_capacity_factor(node,commodity,grade,level_renewable,year)
base_ren_cap_fact = pd.DataFrame({
        'node': country,
        'commodity': 'wind', 
        'level' : 'renewable',       
        'year': model_horizon,
    })
ren_cap_fact1 = make_df(base_ren_cap_fact, grade = 'g1', 
                              value = 1, unit = 'MW')

scen.add_par('renewable_capacity_factor',ren_cap_fact1)

ren_cap_fact2 = make_df(base_ren_cap_fact, grade = 'g2', 
                              value = 0.5, unit = 'MW')
scen.add_par('renewable_capacity_factor',ren_cap_fact2)

In [ ]:
# reliability_factor(node,inv_tec,year,commodity,level,time,rating)
base_reliability_factor = pd.DataFrame({
        'node': country,
        'year_act': model_horizon,
        'commodity': 'electricity',
        'level' : 'final',       
        'time' : 'year',
    })

reliability_factor0 = make_df(base_reliability_factor, technology = 'wind_ppl',
                              rating = 'r0', value = 0.05, unit ='-') # in South Africa example it is 0. 
# but then the technology will never be used
scen.add_par('reliability_factor',reliability_factor0)
reliability_factor1 = make_df(base_reliability_factor, technology = 'wind_ppl',
                              rating = 'r0_4', value = 0.4, unit ='-')
scen.add_par('reliability_factor',reliability_factor1)
reliability_factor2 = make_df(base_reliability_factor, technology = 'wind_ppl',
                              rating = 'r0_8', value = 0.8 , unit ='-')
scen.add_par('reliability_factor',reliability_factor2)
reliability_factor3 = make_df(base_reliability_factor, technology = 'coal_ppl',
                              rating = 'r_full', value = 1 , unit ='-')
scen.add_par('reliability_factor',reliability_factor3)

In [ ]:
# peak_load_factor(node,commodity,level,year,time)
peak_load_factor = pd.DataFrame({
        'node': country,
        'commodity': 'electricity',
        'level' : 'final',       
        'year': model_horizon,
        'time' : 'year',
        'value' : 2,
        'unit' : '-',
    })
scen.add_par('peak_load_factor',peak_load_factor)

In [ ]:
# rating_bin(node,inv_tec,year,commodity,level,time,rating) only wind? non renewables? should assign = 1 to coal?
base_rating_bin = pd.DataFrame({
        'node': country,
        'technology' : 'wind_ppl',
        'year_act': model_horizon,
        'commodity': 'electricity',
        'level' : 'final',        
        'time' : 'year',
    })

rating_bin1 = make_df(base_rating_bin,  rating = 'r0', value = 0.7 , unit ='-')
scen.add_par('rating_bin',rating_bin1)
rating_bin2 = make_df(base_rating_bin, rating = 'r0_4', value = 0.1 , unit ='-')
scen.add_par('rating_bin',rating_bin2)
rating_bin3 = make_df(base_rating_bin, rating = 'r0_8', value = 0.2 , unit ='-')
scen.add_par('rating_bin',rating_bin3)

In [ ]:
# flexibility_factor(node,tec,vintage,year,mode,commodity,level,time,rating)
base_flexibility_factor = pd.DataFrame({
        'node_loc': country,
        'year_vtg': vintage_years,
        'year_act': act_years,
        'mode' : 'standard',
        'commodity': 'electricity',
        'level' : 'final',       
        'time' : 'year',
    })

flexibility_factor0 = make_df(base_flexibility_factor, technology = 'wind_ppl',
                              rating = 'r0', value = -0.2, unit ='-') # in South Africa example it is 0. 
# but then the technology will never be used
scen.add_par('flexibility_factor',flexibility_factor0)
flexibility_factor1 = make_df(base_flexibility_factor, technology = 'wind_ppl',
                              rating = 'r0_4', value = -0.35, unit ='-')
scen.add_par('flexibility_factor',flexibility_factor1)
flexibility_factor2 = make_df(base_flexibility_factor, technology = 'wind_ppl',
                              rating = 'r0_8', value = -0.5 , unit ='-')
scen.add_par('flexibility_factor',flexibility_factor2)
flexibility_factor3 = make_df(base_flexibility_factor, technology = 'coal_ppl',
                              rating = 'r_full', value = 0.1 , unit ='-')
scen.add_par('flexibility_factor',flexibility_factor3)

### define new variables and equations

In [ ]:
scen.init_var('ACT_FLEX', idx_sets=['node','technology','year_vtg','year','mode','time','rating'] )
scen.init_equ( "OPERATING_RESERVE_CONSTRAINT", idx_sets=['node','year','commodity','level','time'] )
scen.init_equ('FLEXIBILITY_PROVISION', idx_sets=['node','technology','year_vtg','year','mode','time'] )
scen.init_equ('FLEX_ACT_CAP_RELATION', idx_sets=['node','technology','year_vtg','year','time','rating'] )

### commit and solve

In [ ]:
scen.commit(comment='define parameters for renewable implementation')
scen.set_as_default()

In [ ]:
scen.solve('MESSAGE')

In [ ]:
scen.var('OBJ')['lvl']

### plotting

In [ ]:
from tools import Plots
p = Plots(scen, country, firstyear=model_horizon[0])

In [ ]:
p.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

In [ ]:
p.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

In [ ]:
p.plot_prices(subset=['light'], baseyear=True)

In [ ]:
mp.close_db()

## 